In [12]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds

In [13]:
# Load the AFLW2K dataset
ds = tfds.load('aflw2k3d', split='train')

for ex in ds.take(4):
    print(ex['image'])

# Here we will split the image data and its corresponding 2D landmark coordinates.
images = []
landmarks2D = []

for ex in ds.take(2000):
    # extracts the image data from the current ex
    images.append(ex['image'])
    # extracts the 2D landmark coordinates
    landmarks2D.append(ex['landmarks_68_3d_xy_normalized'])

# convert the lists to numpy arrays
images = np.array(images)
landmarks2D = np.array(landmarks2D)

# print the length of the image array 
print (len(images))
# print the shape of the 2D landmark coordinates
print (landmarks2D[1].shape)

tf.Tensor(
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]], shape=(450, 450, 3), dtype=uint8)
tf.Tensor(
[[[  9   0   0]
  [ 10   1   0]
  [ 13   4   0]
  ...
  [  4   5   0]
  [  5   5   0]
  [  5   5   0]]

 [[  9   0   0]
  [ 10   1   0]
  [ 14   5   0]
  ...
  [  4   5   0]
  [  4   4   0]
  [  4   4   0]]

 [[ 12   3   0]
  [ 13   5   0]
  [ 17   9   0]
  ...
  [  5   6   0]
  [  5   5   0]
  [  5   5   0]]

 ...

 [[190 157 114]
  [191 158 115]
  [186 154 113]
  ...
  [ 53  35  13]
  [ 55  37  13]
  [ 55  38  12]]

 [[194 163 117]
  [193 165 118]
  [188 161 118]
  ...
  [ 52  36  13]
  [ 53  37  12]
  [ 53  37  12]]

 [[

In [14]:
# Split the dataset into training and testing sets

# Shuffle indices
num_examples = len(images)
# create an array of integers ranging from 0 to num_examples to represent the indices of the data set
indices = np.arange(num_examples)
# shuffle these indices 
np.random.shuffle(indices)

# Define split sizes
train_size = 1500
# 2000-1500=500
test_size = num_examples - train_size 

print(train_size, "\n", test_size)

# Split indices: some for train data (1500) and some for test data(500)
train_indices = indices[:train_size]
test_indices = indices[train_size:]

# Split the dataset
train_images, test_images = images[train_indices], images[test_indices]

train_landmarks, test_landmarks = landmarks2D[train_indices], landmarks2D[test_indices]

1500 
 500


In [15]:
# Build the CNN model
# Define the model outside of a tf.function decorated function
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(450, 450, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(136, activation='relu'),  # Output layer for 68 landmarks (68 * 2 = 136)
])


c:\Users\Heba\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [16]:
# Compile the model
# Mean Squared Error = mse the loss function
# Mean Absolute Error = mae measures the average absolute difference between the predicted and actual values.

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [17]:
# Reshape the target values to match the output shape of the model
train_landmarks_reshaped = train_landmarks.reshape(train_landmarks.shape[0], -1)

# train the model 
history = model.fit(train_images, train_landmarks_reshaped, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 120s 3s/step - loss: 300.0565 - mae: 2.9078 - val_loss: 0.3210 - val_mae: 0.5528
Epoch 2/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 124s 3s/step - loss: 0.3211 - mae: 0.5530 - val_loss: 0.3210 - val_mae: 0.5528
Epoch 3/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 122s 3s/step - loss: 0.3206 - mae: 0.5527 - val_loss: 0.3210 - val_mae: 0.5528
Epoch 4/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 118s 3s/step - loss: 0.3212 - mae: 0.5530 - val_loss: 0.3210 - val_mae: 0.5528
Epoch 5/10
35/43 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - loss: 0.3212 - mae: 0.5530

In [ ]:
#the number of landmarks
num_landmarks = model.output_shape[1] // 2  # Divide by 2 because each landmark has x and y coordinates

In [ ]:
# Get the lowest training loss from the history
lowest_loss = min(history.history['loss'])

In [ ]:
# Reshape test_landmarks to match the output shape of the model
test_landmarks_reshaped = test_landmarks.reshape(test_landmarks.shape[0], -1)

# Evaluate the model
test_loss, test_mae = model.evaluate(test_images, test_landmarks_reshaped)

16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 620ms/step - loss: 0.3214 - mae: 0.5531


In [ ]:
print("Test Loss:", test_loss)
print("Test MAE:", test_mae)

Test Loss: 0.3214515447616577
Test MAE: 0.5532915592193604


In [ ]:
# Print the number of landmarks and lowest loss
print("Number of landmarks:", num_landmarks)
print("Lowest training loss:", lowest_loss)

Number of landmarks: 68
Lowest training loss: 0.32063522934913635
